In [1]:
import pickle

import ortools.sat.python.cp_model as cp_model

In [2]:
with open("./simple_model_mem_alloc_data.pkl", "rb") as fid:
    mem_alloc_data = pickle.load(fid)

In [3]:
mem_alloc_data

{'tensors_meta': {'input_1:0': ([1, 28, 28, 1], dtype('float32'), 4),
  'input_1_int8:0': ([1, 28, 28, 1], dtype('int8'), 1),
  'StatefulPartitionedCall/my_model_1/conv2d_1/Conv2D/ReadVariableOp:0': ([1,
    3,
    3,
    32],
   dtype('int8'),
   1),
  'StatefulPartitionedCall/my_model_1/conv2d_1/Conv2D_bias:0': ([32],
   dtype('int32'),
   4),
  'StatefulPartitionedCall/my_model_1/conv2d_1/Relu:0': ([1, 26, 26, 32],
   dtype('int8'),
   1),
  'StatefulPartitionedCall/my_model_1/max_pooling2d_1/MaxPool:0': ([1,
    13,
    13,
    32],
   dtype('int8'),
   1),
  'StatefulPartitionedCall/my_model_1/max_pooling2d_1/MaxPool_0_Reshape0:0': ([1,
    5408],
   dtype('int8'),
   1),
  'StatefulPartitionedCall/my_model_1/dense_2/MatMul/ReadVariableOp/transpose:0': ([5408,
    128],
   dtype('int8'),
   1),
  'StatefulPartitionedCall/my_model_1/dense_2/MatMul_bias:0': ([128],
   dtype('int32'),
   4),
  'StatefulPartitionedCall/my_model_1/dense_2/Relu:0': ([1, 128],
   dtype('int8'),
   1),
  